In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="wBkRsBb91oiGLAMDrvXu")
project = rf.workspace("padelballdataset").project("padel_ball")
version = project.version(3)
dataset = version.download("coco")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to padel_ball-3 in coco:: 100%|██████████| 5458/5458 [00:01<00:00, 3951.45it/s]


In [3]:
!mkdir padel_ball-3/train/sport_ball
!mkdir padel_ball-3/valid/sport_ball

!mv padel_ball-3/train/* padel_ball-3/train/sport_ball
!mv padel_ball-3/valid/* padel_ball-3/valid/sport_ball

mv: cannot move 'padel_ball-3/train/sport_ball' to a subdirectory of itself, 'padel_ball-3/train/sport_ball/sport_ball'
mv: cannot move 'padel_ball-3/valid/sport_ball' to a subdirectory of itself, 'padel_ball-3/valid/sport_ball/sport_ball'


In [4]:
!rm -R /content/padel_ball-3/valid/.ipynb_checkpoints
!rm -R /content/padel_ball-3/train/.ipynb_checkpoints

rm: cannot remove '/content/padel_ball-3/valid/.ipynb_checkpoints': No such file or directory
rm: cannot remove '/content/padel_ball-3/train/.ipynb_checkpoints': No such file or directory


In [5]:
import os
import torch
from torchvision import datasets, transforms

# Define valid extensions (you can adjust if needed)
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')

# Function to check if a file is valid (ignores hidden directories and files)
def is_valid_file(file_path):
    return file_path.endswith(valid_extensions) and not os.path.basename(file_path).startswith('.')

# Define your transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets, applying the valid file filter
train_dataset = datasets.ImageFolder(root='/content/padel_ball-3/train', transform=transform, is_valid_file=is_valid_file)
val_dataset = datasets.ImageFolder(root='/content/padel_ball-3/valid', transform=transform, is_valid_file=is_valid_file)

# Create DataLoaders for training and validation sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)


In [6]:
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the ResNet model
import torchvision.models.detection as detection
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 1000  # 1 class for tennis ball + 1 for background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


# Move model to GPU if available
model = model.to(device)

# Define loss and optimizer
import torch.optim as optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 154MB/s]


In [7]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs=10
# for epoch in range(epochs):
#     model.train()
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()


In [8]:
import torch

def validate_model(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # For accuracy calculation:
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = val_loss / len(val_loader)  # Average validation loss
    accuracy = 100 * correct / total       # Validation accuracy

    return avg_loss, accuracy


In [9]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    # Training Loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation Step
    val_loss, val_accuracy = validate_model(model, val_loader, criterion)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Early stopping logic can go here


Epoch 1, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 2, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 3, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 4, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 5, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 6, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 7, Validation Loss: 0.0000, Validation Accuracy: 100.00%
Epoch 8, Validation Loss: 0.0000, Validation Accuracy: 100.00%


KeyboardInterrupt: 

In [17]:
 !ls /content/padel_ball-3/train/'sport ball'


Streaming output truncated to the last 5000 lines.
162_png.rf.e02b85f10aca9f1f6b22035802505b48.jpg
162_png.rf.f93513b448170dccba86b60dddc4790e.jpg
168_png.rf.4f501117d9296997a609af9bd75a0bbf.jpg
168_png.rf.a88838a89f4a734c48cb646ae07b19fe.jpg
168_png.rf.f6cc3b795774d6e448b9448bf7fc9d60.jpg
16_png.rf.98fa60cf13e8969fd108d7b97e06d238.jpg
16_png.rf.b2ca2024deff32842bb2bc5a5bf9ca70.jpg
16_png.rf.c23ac132d4804bfc69adb0c79b0e1f8c.jpg
18_png.rf.030e1392ad0ae87c946f32a70d1b8088.jpg
18_png.rf.3cdc21c9820d9f38c56fbb1c93bc47c1.jpg
18_png.rf.8e15529ab7d1acd99dc6abd114671e46.jpg
19_png.rf.783bcd3fab3ce82e17b88af0698cdd1f.jpg
19_png.rf.c2cccadd2174f71be00b5a7071a18d96.jpg
19_png.rf.f86cea7d467ceb84071369f8600d112a.jpg
20_png.rf.2949ec056208c06d36b81f7e028c268f.jpg
20_png.rf.67009adce939fb4431dfa3076c5d4538.jpg
20_png.rf.dcb4ae71aebf795cc76eb812a35efb14.jpg
225_png.rf.42afd3b5bb8e606e1a95ae624fc14ee2.jpg
225_png.rf.9709f7fa90d490ec57c0376093e2e0ff.jpg
225_png.rf.c92e32a6c6d55a03e9a3d0cae3eaaee8.jpg
2

# NEW TESTS

In [3]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [6]:
!ls /content/padel_ball-3/valid/*.json

/content/padel_ball-3/valid/_annotations.coco.json


In [23]:
from torchvision.transforms import ToTensor

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_transform():
    return ToTensor()

# Load datasets
train_dataset = CocoDetection(root='/content/padel_ball-3/train', annFile='/content/padel_ball-3/train/_annotations.coco.json', transform=get_transform())
val_dataset = CocoDetection(root='/content/padel_ball-3/valid', annFile='/content/padel_ball-3/valid/_annotations.coco.json', transform=get_transform())

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [24]:
# Custom collate function

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

In [12]:

# Load pre-trained Faster R-CNN model with ResNet50 backbone
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 177MB/s]


In [25]:
# Modify the model to fit your classes
num_classes = 2  # 1 class for tennis ball + 1 for background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define optimizer and scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [31]:
# Define optimizer and learning rate
from torch.optim import SGD

# Define optimizer and learning rate
optimizer = SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Function to convert COCO-style annotations to the format expected by Faster R-CNN
def coco_target_to_fastrcnn_target(coco_target):
    boxes = []
    labels = []

    for obj in coco_target:
        x_min, y_min, width, height = obj['bbox']

        # Ensure width and height are positive
        if width > 0 and height > 0:
            boxes.append([x_min, y_min, x_min + width, y_min + height])
            labels.append(obj['category_id'])

    return {
        'boxes': torch.tensor(boxes, dtype=torch.float32),
        'labels': torch.tensor(labels, dtype=torch.int64)
    }


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, targets in train_loader:
        images = [image.to(device) for image in images]

        # Convert the COCO targets to the format expected by Faster R-CNN
        targets = [coco_target_to_fastrcnn_target(t) for t in targets]

        # Skip images without valid bounding boxes
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets if t['boxes'].shape[0] > 0]
        images = [img for img, t in zip(images, targets) if t['boxes'].shape[0] > 0]

        if len(images) == 0:
            # Skip the batch if no valid images remain
            continue

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss}")

# Save the trained model
torch.save(model.state_dict(), 'fasterrcnn_resnet50_tennis_ball.pth')


Epoch 1, Loss: 448.9494778588414
Epoch 2, Loss: 376.7752861902118
Epoch 3, Loss: 347.3139609694481
Epoch 4, Loss: 336.4934106618166
Epoch 5, Loss: 327.86623176932335
Epoch 6, Loss: 318.8271893635392
Epoch 7, Loss: 308.2134248316288
Epoch 8, Loss: 299.9593310132623
Epoch 9, Loss: 292.5972136706114
Epoch 10, Loss: 304.2716890797019


In [36]:
def evaluate_model(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient computation
        for images, targets in data_loader:
            images = [image.to(device) for image in images]

            # Forward pass (for inference/prediction)
            predictions = model(images)

            # Process predictions (e.g., print them or calculate metrics)
            for prediction in predictions:
                print(prediction)  # Replace with actual evaluation code

# Assuming your model is loaded and trained
evaluate_model(model, val_loader, device)


{'boxes': tensor([[521.6609, 453.4537, 527.1404, 462.6002],
        [384.6264, 399.0880, 389.1236, 405.3197],
        [336.8807, 314.1121, 341.6268, 320.8177],
        [268.0071, 442.1473, 276.6512, 458.5908],
        [470.0118, 302.3186, 476.3623, 313.2517],
        [383.0303, 399.8705, 387.6026, 405.2167],
        [519.6237, 450.8506, 529.7936, 465.1890],
        [180.0936, 296.1155, 184.8932, 304.9395],
        [383.0632, 398.2613, 387.3305, 403.5002],
        [383.7786, 397.0122, 388.6384, 403.9179],
        [265.5010, 436.7973, 278.8037, 461.3365]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), 'scores': tensor([0.9644, 0.8973, 0.8879, 0.8514, 0.4859, 0.1578, 0.1428, 0.0651, 0.0649,
        0.0610, 0.0520], device='cuda:0')}
{'boxes': tensor([[521.5635, 453.4705, 527.0134, 462.7343],
        [385.3429, 398.9071, 389.4166, 405.3829],
        [340.3550, 409.0082, 344.4746, 414.7535],
        [341.8457, 380.2495, 345.3345, 384.7613],
        

## detection

In [37]:
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 100.7 MB/s eta 0:00:00


In [45]:
import torch
import cv2
import numpy as np
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision import models
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort
import csv
import math
import time

# COCO dataset class labels
# Custom dataset class labels (background and tennis ball)
CUSTOM_INSTANCE_CATEGORY_NAMES = ['__background__', 'sports ball']


# Check if GPU is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device:", device)

# Load a pretrained Faster R-CNN model with ResNet50 backbone
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load the pre-trained Faster R-CNN model
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

# Get the number of classes (from your fine-tuning setup, 2: background and tennis ball)
num_classes = 2

# Replace the box predictor with a new one (for 2 classes)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the fine-tuned model weights
model.load_state_dict(torch.load('/content/fasterrcnn_resnet50_tennis_ball.pth'))
model.eval()
model = model.to(device)


# Initialize DeepSort
tracker = DeepSort(max_age=75, n_init=3, nms_max_overlap=1.0)

# Video paths
input_video_path = '/content/20240725T203000_Trimm.mp4'  # Change to your video path
output_video_path = '20240724T191501_output_video.mp4'

# Load the video
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize variables for tracking
previous_positions = {}
total_distance_travelled = {}
ball_angles = {}
csv_data = []

# Assume 1 pixel = 1/154 meters
meters_per_pixel = 1 / 154

# Process the video in batches of 25 frames
batch_size = 2
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Dictionary to store frame timestamps
frame_timestamps = {}

with torch.no_grad():
    for batch_start in tqdm(range(0, total_frames, batch_size), desc="Processing Video"):
        batch_end = min(batch_start + batch_size, total_frames)

        # Initialize variables for the batch
        batch_images = []
        batch_detections = []

        # Read and process frames in the batch
        for frame_idx in range(batch_start, batch_end):
            ret, frame = cap.read()
            if not ret:
                break

            # Capture timestamp for each frame
            frame_timestamps[frame_idx] = time.time()

            # Convert frame to tensor
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_tensor = F.to_tensor(image).unsqueeze(0).to(device)
            batch_images.append(image_tensor)

        # Check if batch_images is not empty before concatenating
        if len(batch_images) > 0:
            # Perform object detection on the batch
            batch_images = torch.cat(batch_images, dim=0)
            batch_predictions = model(batch_images)

            # Process the batch predictions
            for i in range(len(batch_predictions)):
                frame_idx = batch_start + i
                predictions = batch_predictions[i]
                boxes = predictions['boxes'].cpu().numpy()
                labels = predictions['labels'].cpu().numpy()
                scores = predictions['scores'].cpu().numpy()

                # Filter labels for "person", "sports ball", and "tennis racket"
                filtered_indices = []
                for j, label in enumerate(labels):
                  if label < len(CUSTOM_INSTANCE_CATEGORY_NAMES):
                    if CUSTOM_INSTANCE_CATEGORY_NAMES[label] == 'sports ball':
                      filtered_indices.append(j)
                # for j, label in enumerate(labels):
                #     if label < len(COCO_INSTANCE_CATEGORY_NAMES):
                #         if COCO_INSTANCE_CATEGORY_NAMES[label] in ['person', 'sports ball', 'tennis racket']:
                #             filtered_indices.append(j)
                    else:
                        print(f"Warning: Label {label} is out of range for COCO_INSTANCE_CATEGORY_NAMES.")

                filtered_boxes = boxes[filtered_indices]
                filtered_scores = scores[filtered_indices]
                filtered_labels = labels[filtered_indices]

                # Prepare detections for DeepSort
                detections = []
                for j in range(len(filtered_boxes)):
                    if filtered_scores[j] > 0.5:
                        x1, y1, x2, y2 = filtered_boxes[j]
                        bbox = [x1, y1, x2 - x1, y2 - y1]
                        detections.append((bbox, filtered_scores[j], filtered_labels[j]))

                batch_detections.append(detections)

        # Update tracker with current detections
        for i, detections in enumerate(batch_detections):
            frame_idx = batch_start + i
            ret, frame = cap.read()
            if not ret:
                break

            # Ensure frame is read at the correct timestamp
            while time.time() < frame_timestamps[frame_idx]:
                time.sleep(0.001)

            tracked_objects = tracker.update_tracks(detections, frame=frame)

            # Draw bounding boxes and labels on the frame
            for obj in tracked_objects:
                if not obj.is_confirmed():
                    continue

                x1, y1, x2, y2 = map(int, obj.to_tlbr())
                #label = COCO_INSTANCE_CATEGORY_NAMES[obj.det_class]
                label = CUSTOM_INSTANCE_CATEGORY_NAMES[obj.det_class]  # Custom labels

                track_id = obj.track_id

                # Convert pixel coordinates to meters
                x_meters = ((x1 + x2) / 2) * meters_per_pixel
                y_meters = ((y1 + y2) / 2) * meters_per_pixel
                z_meters = 1.0 if label == 'sports ball' else 0.0

                # Calculate the ball speed if it's a sports ball
                if label == 'sports ball':
                    current_position = (x_meters, y_meters)
                    if track_id in previous_positions:
                        prev_position = previous_positions[track_id]
                        distance_meters = math.sqrt((current_position[0] - prev_position[0]) ** 2 +
                                                    (current_position[1] - prev_position[1]) ** 2)
                        speed_m_per_s = distance_meters * fps
                        speed_km_per_h = speed_m_per_s * 3.6

                        # Calculate angle of movement
                        dx = current_position[0] - prev_position[0]
                        dy = current_position[1] - prev_position[1]
                        angle = math.degrees(math.atan2(dy, dx))
                        if angle < 0:
                            angle += 360

                        ball_angles[track_id] = angle

                        # Determine trajectory
                        trajectory = "down the line"
                        if 0 <= angle < 20 or 120 <= angle < 180:
                            trajectory = "cross court"

                    else:
                        speed_km_per_h = 0
                        trajectory = "down the line"

                    previous_positions[track_id] = current_position

                # Calculate distance traveled by players
                if label == 'person':
                    current_position = (x_meters, y_meters)
                    if track_id in previous_positions:
                        prev_position = previous_positions[track_id]
                        distance_meters = math.sqrt((current_position[0] - prev_position[0]) ** 2 +
                                                    (current_position[1] - prev_position[1]) ** 2)
                        total_distance_travelled[track_id] = total_distance_travelled.get(track_id, 0) + distance_meters

                    previous_positions[track_id] = current_position

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Prepare text and background for ID
                text = f'{label} ID: {track_id}'
                (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)

                # Draw the background rectangle for the text
                cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), (0, 255, 0), -1)  # Filled rectangle

                # Draw the text on top of the background
                cv2.putText(frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

                if label == 'sports ball':
                    speed_text = f'Speed: {speed_km_per_h:.2f} km/h'
                    (speed_text_width, speed_text_height), _ = cv2.getTextSize(speed_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)


                    # Draw background for speed text
                    cv2.rectangle(frame, (x1, y2 + 5), (x1 + speed_text_width, y2 + speed_text_height + 15), (255, 0, 0), -1)

                    # Draw the speed text
                    cv2.putText(frame, speed_text, (x1, y2 + speed_text_height + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    trajectory_text = f'Trajectory: {trajectory}'
                    (trajectory_text_width, trajectory_text_height), _ = cv2.getTextSize(trajectory_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)

                    # Draw background for trajectory text
                    cv2.rectangle(frame, (x1, y2 + speed_text_height + 20), (x1 + trajectory_text_width, y2 + speed_text_height + trajectory_text_height + 30), (255, 0, 0), -1)

                    # Draw the trajectory text
                    cv2.putText(frame, trajectory_text, (x1, y2 + speed_text_height + trajectory_text_height + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                # Store the information in CSV data
                csv_data.append([frame_idx, track_id, label, x_meters, y_meters, z_meters,
                                 speed_km_per_h if label == 'sports ball' else '',
                                 total_distance_travelled.get(track_id, '') if label == 'person' else '',
                                 trajectory if label == 'sports ball' else ''])

            # Write the processed frame to the output video
            out.write(frame)

# Write CSV data to file
with open('20240724T191501_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Frame', 'ID', 'Label', 'X (m)', 'Y (m)', 'Z (m)', 'Speed (km/h)', 'Distance Traveled (m)', 'Trajectory'])
    writer.writerows(csv_data)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

Using device: cuda


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Processing Video: 100%|██████████| 461/461 [01:58<00:00,  3.89it/s]


In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in train_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    lr_scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_iou = 0
        num_boxes = 0

        for images, targets in val_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs = model(images)

            # Evaluate outputs and targets
            for output, target in zip(outputs, targets):
                if len(output['boxes']) > 0:  # Check if there are any predictions
                    pred_boxes = output['boxes']
                    pred_scores = output['scores']
                    pred_labels = output['labels']

                    # Filter out low confidence predictions (e.g., score > 0.5)
                    high_conf_indices = pred_scores > 0.5
                    pred_boxes = pred_boxes[high_conf_indices]
                    pred_labels = pred_labels[high_conf_indices]

                    # Calculate IoU for each predicted box with ground truth boxes
                    target_boxes = target['boxes']
                    if target_boxes.size(0) > 0:
                        iou = box_iou(pred_boxes, target_boxes)
                        total_iou += iou.sum().item()  # Sum IoUs for all predictions
                        num_boxes += pred_boxes.size(0)  # Count the number of predicted boxes

        # Calculate average IoU
        if num_boxes > 0:
            avg_iou = total_iou / num_boxes
        else:
            avg_iou = 0.0

        print(f'Epoch {epoch + 1}, Average IoU: {avg_iou:.4f}')

AttributeError: 'list' object has no attribute 'items'